<a href="https://colab.research.google.com/github/d556645fw/Side-project/blob/main/Lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf /content/diffusers
!git clone https://github.com/huggingface/diffusers.git


In [ ]:
!pip install -r ./diffusers/examples/text_to_image/requirements.txt

In [ ]:
!huggingface-cli login

In [ ]:
!export MODEL_NAME="CompVis/stable-diffusion-v1-4"
!export DATASET_NAME="lambdalabs/naruto-blip-captions"

!accelerate launch --mixed_precision="fp16"  train_text_to_image.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --dataset_name=$DATASET_NAME \
  --use_ema \
  --resolution=512 --center_crop --random_flip \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --gradient_checkpointing \
  --max_train_steps=15000 \
  --learning_rate=1e-05 \
  --max_grad_norm=1 \
  --lr_scheduler="constant" --lr_warmup_steps=0 \
  --output_dir="sd-naruto-model"

In [ ]:
!pip install diffusers

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

model_path = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
pipe.to("cuda")

image = pipe(prompt="yoda").images[0]
image.save("yoda-naruto.png")

In [ ]:
!mkdir train
!unzip train.zip -d train

In [ ]:
!accelerate launch --mixed_precision="fp16" ./diffusers/examples/text_to_image/train_text_to_image_lora.py \
  --pretrained_model_name_or_path="CompVis/stable-diffusion-v1-4" \
  --dataset_name="./train" --caption_column="text" \
  --resolution=512 --random_flip \
  --train_batch_size=1 \
  --num_train_epochs=100 --checkpointing_steps=5000 \
  --learning_rate=1e-04 --lr_scheduler="constant" --lr_warmup_steps=0 \
  --seed=42 \
  --output_dir="sd-my-model-lora" \
  --validation_prompt="a girl with white hair" --report_to="wandb"\
  --mixed_precision="no"

In [ ]:
!pip uninstall -y diffusers
!git clone https://github.com/huggingface/diffusers.git
!pip install -e ./diffusers

In [ ]:
!pip install -U peft

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_path = "./sd-my-model-lora"
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
pipe.unet.load_attn_procs(model_path)
pipe.to("cuda")

In [ ]:
import matplotlib.pyplot as plt
prompt = "A girl who has blue hair"
image = pipe(prompt, num_inference_steps=100, guidance_scale=8.3).images[0]
plt.imshow(image)